In [35]:
import cv2
import pickle
import os.path
import numpy as np
np.random.seed(0)
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from helpers import resize_to_fit




# Dependencies to Visualize the model# Depen 
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)


In [3]:
letters =  "letters"
model_file = 'captcha_model.hdf5'
model_labels = 'model_labels.dat'
data = []
labels = []

In [4]:
for char_file in paths.list_images(letters):
    #fix images
    image = cv2.imread(char_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = resize_to_fit(image, 20, 20)
    image = np.expand_dims(image, axis=2)
    #Add test and correct data
    label = char_file.split(os.path.sep)[-2]
    data.append(image)
    labels.append(label)
    #make zeros and ones

In [5]:
 #make zeros and ones
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)


In [6]:
#make test and train data
(x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size = 0.25, random_state = 0)
Binarizer = LabelBinarizer().fit(y_train)
y_train = Binarizer.transform(y_train)
y_test = Binarizer.transform(y_test)
with open(model_labels, "wb") as f:
  pickle.dump(Binarizer, f)

In [53]:
#build nerual net
model = Sequential()
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dense(19, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=10, verbose=1)
model.save(model_file)

Train on 71238 samples, validate on 23746 samples
Epoch 1/10
71238/71238 [==============================] - 110s 2ms/step - loss: 0.1786 - acc: 0.9531 - val_loss: 0.0603 - val_acc: 0.9852
Epoch 2/10
71238/71238 [==============================] - 105s 1ms/step - loss: 0.0478 - acc: 0.9878 - val_loss: 0.0480 - val_acc: 0.9880
Epoch 3/10
71238/71238 [==============================] - 101s 1ms/step - loss: 0.0324 - acc: 0.9912 - val_loss: 0.0457 - val_acc: 0.9894
Epoch 4/10
71238/71238 [==============================] - 102s 1ms/step - loss: 0.0249 - acc: 0.9931 - val_loss: 0.0486 - val_acc: 0.9893
Epoch 5/10
71238/71238 [==============================] - 102s 1ms/step - loss: 0.0173 - acc: 0.9950 - val_loss: 0.0541 - val_acc: 0.9888
Epoch 6/10
71238/71238 [==============================] - 103s 1ms/step - loss: 0.0129 - acc: 0.9968 - val_loss: 0.0624 - val_acc: 0.9880
Epoch 7/10
71238/71238 [==============================] - 106s 1ms/step - loss: 0.0142 - acc: 0.9964 - val_loss: 0.0610 - 

In [54]:

model.save(model_file)

In [55]:
model_loss, model_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.06493915398939694, Accuracy: 0.9884191021645751
